In [ ]:
# Step 1: Install necessary packages
!pip install transformers tensorflow datasets

In [ ]:
# Step 2: Import necessary libraries
import tensorflow as tf
from transformers import (
    BertTokenizer,
    TFBertForSequenceClassification,
    create_optimizer
)
import pandas as pd

In [ ]:
# Downloasd dataset

!gdown https://drive.google.com/file/d/1DNgciEZTQWuo_wq4qMGIV8z3VC1Onjjv/view?usp=share_link --fuzzy
!gdown https://drive.google.com/file/d/1ePfUhb2XeicnybAMu0y2w3BhiWm4XY8t/view?usp=share_link --fuzzy
!gdown https://drive.google.com/file/d/1FRn6Gn3ijmCWfhUI6QkGm4g4LeQ7Veg0/view?usp=share_link --fuzzy
!gdown https://drive.google.com/file/d/1Fw3yivW_PSH-C5T_4-9bWshxPHxvxtAi/view?usp=share_link --fuzzy
!gdown https://drive.google.com/file/d/1F9r26d7emj43EG6kcMfvOsu1rI8yxR4d/view?usp=share_link --fuzzy
!gdown https://drive.google.com/file/d/1zSYr8akNxtCCR_TxVIFrel1k254k-vin/view?usp=share_link --fuzzy
!gdown https://drive.google.com/file/d/1zSYr8akNxtCCR_TxVIFrel1k254k-vin/view?usp=share_link --fuzzy
!gdown https://drive.google.com/file/d/1zSYr8akNxtCCR_TxVIFrel1k254k-vin/view?usp=share_link --fuzzy
!gdown https://drive.google.com/file/d/1I0vjwcnwfaMZdwhS-oJjqUoihl9NxZ1g/view?usp=share_link --fuzzy
!gdown https://drive.google.com/file/d/1tX9mZBStZGCQmOhgaCu-tSdlXqQU77ou/view?usp=share_link --fuzzy
!gdown https://drive.google.com/file/d/1-GNcBpiB3Y0RzbJLawxL31IjkTBtpUz2/view?usp=share_link --fuzzy

Downloading...
From: https://drive.google.com/uc?id=1DNgciEZTQWuo_wq4qMGIV8z3VC1Onjjv
To: /content/laptop_dev.csv
100% 46.5k/46.5k [00:00<00:00, 74.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ePfUhb2XeicnybAMu0y2w3BhiWm4XY8t
To: /content/laptop_test.csv
100% 98.6k/98.6k [00:00<00:00, 73.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1FRn6Gn3ijmCWfhUI6QkGm4g4LeQ7Veg0
To: /content/laptop_train.csv
100% 378k/378k [00:00<00:00, 103MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Fw3yivW_PSH-C5T_4-9bWshxPHxvxtAi
To: /content/mams_test.csv
100% 222k/222k [00:00<00:00, 48.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1F9r26d7emj43EG6kcMfvOsu1rI8yxR4d
To: /content/mams_train.csv
100% 1.78M/1.78M [00:00<00:00, 129MB/s]
Downloading...
From: https://drive.google.com/uc?id=1zSYr8akNxtCCR_TxVIFrel1k254k-vin
To: /content/mams_val.csv
100% 219k/219k [00:00<00:00, 49.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1zSYr8akNxtCCR_TxVIFrel1k2

In [ ]:
def load_mams_data(path):
  df = pd.read_csv(path)
  df = df[['text', 'term', 'polarity']]
  return df

def load_laptop_data(path):
  text_ds = []
  term_ds = []
  polarity_ds = []
  with open(path, 'r') as f:
    data = f.read().splitlines()

  terms = []
  polarities = []
  text = ""
  term = ""
  polarity = ""
  for row in data[2:]:
    # We are on new text column
    if len(row) == 1:
      if term != "":
        terms.append(term)
        polarities.append(polarity)
      for i in range(len(terms)):
        text_ds.append(text.strip())
        term_ds.append(terms[i].strip())
        polarity_ds.append(polarities[i])

      text = ""
      term = ""
      polarity = ""
      terms = []
      polarities = []

    else:
      token, label = row.rsplit(',', 1)
      text += " " + token

      if label == 'T-POS':
        label = 'positive'
      elif label == 'T-NEG':
        label = 'negative'
      elif label == 'T-NEU':
        label = 'neutral'
      else:
        if term != "":
          terms.append(term)
          polarities.append(polarity)
        term = ""
        polarity = ""
        continue

      if polarity == label:
        term += " " + token

      else:
        if term != "":
          terms.append(term)
          polarities.append(polarity)

        polarity = label
        term = token

  df = pd.DataFrame()
  df['text'] = text_ds
  df['term'] = term_ds
  df['polarity'] = polarity_ds
  return df

In [ ]:
mams_train = pd.read_csv("/content/mams_train.csv")
mams_train.head()

,text,from,polarity,term,to
0,The decor is not special at all but their food...,4,negative,decor,9
1,The decor is not special at all but their food...,42,positive,food,46
2,The decor is not special at all but their food...,59,positive,prices,65
3,"when tables opened up, the manager sat another...",5,neutral,tables,11
4,"when tables opened up, the manager sat another...",27,negative,manager,34


In [ ]:
laptop_train, laptop_test, laptop_valid = list(map(load_laptop_data, ['laptop_train.csv', 'laptop_test.csv', 'laptop_dev.csv']))

# restaurant dataset
rest_train, rest_test, rest_valid = list(map(load_laptop_data, ['rest16_train.csv', 'rest16_test.csv', 'rest16_dev.csv']))

# mams dataset
mams_train, mams_test, mams_valid = list(map(load_mams_data, ['mams_train.csv', 'mams_test.csv', 'mams_val.csv']))

In [ ]:
mams_train.head()

,text,term,polarity
0,The decor is not special at all but their food...,decor,negative
1,The decor is not special at all but their food...,food,positive
2,The decor is not special at all but their food...,prices,positive
3,"when tables opened up, the manager sat another...",tables,neutral
4,"when tables opened up, the manager sat another...",manager,negative


In [ ]:
# dataset combined, shuffled
train_data = pd.concat([laptop_train, rest_train, mams_train, laptop_test, rest_test, mams_test]).sample(frac = 1).reset_index(drop = True)
valid_data = pd.concat([laptop_valid, rest_valid, mams_valid]).sample(frac = 1).reset_index(drop = True)

In [ ]:
print('*'*30,'data ', '*'*30)
train_data.head()

****************************** data  ******************************


,text,term,polarity
0,"great food, even better price, they also have ...",dj,neutral
1,"Besides the price, here are some other annoyan...",price,negative
2,I have been dining at Narita for 6 years and h...,meal,positive
3,"I went into the buffet style ordering counter,...",potato,positive
4,It runs perfectly .,runs,positive


In [ ]:
train_data = train_data[train_data['polarity'] != 'neutral']

valid_data = valid_data[valid_data['polarity'] != 'neutral']

In [ ]:
print('*'*30,'data ', '*'*30)
train_data.head()

****************************** data  ******************************


,text,term,polarity
1,"Besides the price, here are some other annoyan...",price,negative
2,I have been dining at Narita for 6 years and h...,meal,positive
3,"I went into the buffet style ordering counter,...",potato,positive
4,It runs perfectly .,runs,positive
8,Since the menu is not organized in the way to ...,waiter,negative


In [ ]:
print('*'*30,'data ', '*'*30)
valid_data.head()

****************************** data  ******************************


,text,term,polarity
0,Another favorite is the hanger steak which is ...,amount of arugula,positive
2,The owner addressed me by my first name after ...,garlic shrimp,positive
3,It's not going to win any awards for its decor...,food,positive
4,"I would recommend Roxy 's for that "","" but not...",food,negative
5,"That being said, they do have a nice lounge ar...",waiting,negative


In [ ]:
# data processing, encoding labels, and lower casing the sentence text and aspect text

pol2idx = { 'positive' : 1, 'negative' : 0}
for data in [train_data, valid_data]:
  data['polarity'] = data['polarity'].apply(lambda x:pol2idx[x])
  data['text'] = data['text'].apply(lambda x: x.lower())
  data['term'] = data['term'].apply(lambda x: str(x).lower()).astype('str')

  data = data.drop_duplicates(["text", "term", "polarity"])

train_data.head()

,text,term,polarity
1,"besides the price, here are some other annoyan...",price,0
2,i have been dining at narita for 6 years and h...,meal,1
3,"i went into the buffet style ordering counter,...",potato,1
4,it runs perfectly .,runs,1
8,since the menu is not organized in the way to ...,waiter,0


In [ ]:
# Step 4: Initialize tokenizer and encode dataset
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
# Function to tokenize sentences
def encode_sentence(sentence, aspect):
    text = f"[CLS] {aspect} [SEP] {sentence} [SEP]"
    return tokenizer.encode_plus(
        text,
        add_special_tokens=False,
        max_length=100,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='tf'
    )

In [ ]:

# Prepare TensorFlow dataset
def encode_examples(df):
    input_ids_list, attention_mask_list, label_list = [], [], []
    for _, row in df.iterrows():
        encoded = encode_sentence(row['text'], row['term'])
        input_ids_list.append(encoded['input_ids'])
        attention_mask_list.append(encoded['attention_mask'])
        label_list.append(row['polarity'])

    return (
        tf.concat(input_ids_list, axis=0),
        tf.concat(attention_mask_list, axis=0),
        tf.convert_to_tensor(label_list)
    )

In [ ]:
train_input_ids, train_attention_masks, train_labels = encode_examples(train_data)
val_input_ids, val_attention_masks, val_labels = encode_examples(valid_data)


In [ ]:
train_labels

<tf.Tensor: shape=(10829,), dtype=int32, numpy=array([0, 1, 1, ..., 1, 1, 1], dtype=int32)>

In [ ]:
# Step 5: Create TensorFlow Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((
    {'input_ids': train_input_ids, 'attention_mask': train_attention_masks},
    train_labels
)).shuffle(1000).batch(16)


In [ ]:
val_dataset = tf.data.Dataset.from_tensor_slices((
    {'input_ids': val_input_ids, 'attention_mask': val_attention_masks},
    val_labels
)).batch(16)

In [ ]:
# Step 6: Load BERT model with classification head
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Step 7: Compile the model
optimizer, schedule = create_optimizer(init_lr=2e-5, num_train_steps=5000, num_warmup_steps=500)

model.compile(optimizer=optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [ ]:
# Step 8: Train the model
model.fit(train_dataset, epochs=3, validation_data=val_dataset)

Epoch 1/3
677/677 [==============================] - 291s 359ms/step - loss: 0.4519 - accuracy: 0.7758 - val_loss: 0.2949 - val_accuracy: 0.8838
Epoch 2/3
677/677 [==============================] - 228s 337ms/step - loss: 0.1799 - accuracy: 0.9316 - val_loss: 0.2628 - val_accuracy: 0.9142
Epoch 3/3
677/677 [==============================] - 229s 338ms/step - loss: 0.0851 - accuracy: 0.9720 - val_loss: 0.2962 - val_accuracy: 0.9161


In [ ]:
# Step 9: Save fine-tuned model
model.save_pretrained('fine_tuned_bert_absa')
tokenizer.save_pretrained('fine_tuned_bert_absa')

('fine_tuned_bert_absa/tokenizer_config.json',
 'fine_tuned_bert_absa/special_tokens_map.json',
 'fine_tuned_bert_absa/vocab.txt',
 'fine_tuned_bert_absa/added_tokens.json')

In [ ]:
# Step 10: Load the fine-tuned model and tokenizer
loaded_tokenizer = BertTokenizer.from_pretrained('fine_tuned_bert_absa')
loaded_model = TFBertForSequenceClassification.from_pretrained('fine_tuned_bert_absa')

Some layers from the model checkpoint at fine_tuned_bert_absa were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at fine_tuned_bert_absa.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [ ]:
# Step 13: Make predictions on sample data
def predict_sentiment(sentence, aspect):
    encoded = loaded_tokenizer.encode_plus(
        f"[CLS] {aspect.lower()} [SEP] {sentence.lower()} [SEP]",
        add_special_tokens=False,
        max_length=100,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='tf'
    )
    input_ids = encoded['input_ids']
    attention_mask = encoded['attention_mask']

    predictions = loaded_model({'input_ids': input_ids, 'attention_mask': attention_mask})[0]
    predicted_class = tf.argmax(predictions, axis=1).numpy()[0]

    idx2pol = {0: 'negative', 1: 'positive'}
    return idx2pol[predicted_class]

# Example predictions
sample_sentence1 = "The food was amazing, but the service was slow."
sample_aspect1 = "food"
sample_aspect2 = "service"

In [ ]:
print(f"\nSentence: '{sample_sentence1}'")
print(f"Aspect: '{sample_aspect1}' -> Predicted Sentiment: {predict_sentiment(sample_sentence1, sample_aspect1)}")
print(f"Aspect: '{sample_aspect2}' -> Predicted Sentiment: {predict_sentiment(sample_sentence1, sample_aspect2)}")


Sentence: 'The food was amazing, but the service was slow.'
Aspect: 'food' -> Predicted Sentiment: positive
Aspect: 'service' -> Predicted Sentiment: negative


In [ ]:
sample_sentence2 = "The battery life is terrible on this laptop."
sample_aspect3 = "battery life"
print(f"\nSentence: '{sample_sentence2}'")
print(f"Aspect: '{sample_aspect3}' -> Predicted Sentiment: {predict_sentiment(sample_sentence2, sample_aspect3)}")


Sentence: 'The battery life is terrible on this laptop.'
Aspect: 'battery life' -> Predicted Sentiment: negative


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Assume y_true are the actual labels, y_pred are the predicted labels,
# and y_logits are the raw output logits from BERT.

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

# For ROC-AUC, use probabilities or logits converted to probabilities
probabilities = tf.nn.softmax(y_logits, axis=-1)[:, 1]  # Probabilities for positive class
roc_auc = roc_auc_score(y_true, probabilities)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")
